# Calculate the central magnification of an eye - camera system

This integration demonstrates how PAROS can be used to calculate the central magnification of a fundus image.  
More information can be found at [our GitHub repository](https://github.com/MREYE-LUMC/PAROS).

In [1]:
%%html
<style>
.widget-label { min-width: 30ex !important};

tr:nth-child(even) {{
    background-color: #fff;
}}
tr:nth-child(odd) {{
    background-color: #fff;
}}
</style>

In [2]:
# The  simple example, code very close to calculate_ocular_magnification.ipynb,
# but built into a elementary GUI using ipywidgets
# for this, all code needs to be in one notebook, therefore fundscale.py is copied here

from __future__ import annotations

from dataclasses import dataclass

import ipywidgets as widgets
import sympy as sp

from IPython.display import HTML, Markdown

from PAROS.fundscale import Camera, Eye, PhakicIOL, calculate_magnification

In [3]:
# define camera
camera_f_cond = 0.026347
camera_a1 = -0.010541

camera = Camera(F_cond=camera_f_cond, a1=camera_a1)
camera_matrix = sp.simplify(camera.ray_transfer_matrix)

In [4]:
# initialize dataclass for magnificationdata
@dataclass
class MagnificationData:
    magnification: float = None
    lens_back_curvature: float = None
    glasses_curvature: float = None
    glasses_power: float = None
    eye_model: Eye = None

    @classmethod
    def calculate_from_eye_model(cls, eye: Eye, refraction: float):
        """Calculate magnification data from an eye model.

        Parameters
        ----------
        eye : fundScale.Eye
            Eye model.
        refraction : float
            Spherical equivalent of refraction of the eye.

        Returns
        -------
        MagnificationData
            Magnification data of the eye.
        """
        lens_back_curvature, glasses_curvature = eye.adjust_lens_back(
            refraction, update_model=True
        )
        magnification, glasses_power, _ = calculate_magnification(eye, camera)

        return cls(
            magnification, lens_back_curvature, glasses_curvature, glasses_power, eye
        )

    def __str__(self):
        return (
            f"Magnification eye model               : {abs(self.magnification):.2f}\n"
            f"Necessary correction at camera phakic : {self.glasses_power:.2f} D\n"
            f"Eye model                             : {self.eye_model}"
        )

    def to_html(self) -> HTML:
        return HTML(
            f"""
<table>
    <tr>
        <td style="text-align: left">Magnification eye model</td>
        <td>:</td>
        <td style="text-align: left">{abs(self.magnification):.2f}</td>
   </tr>
   <tr>
        <td style="text-align: left">Necessary correction at camera phakic</td>
        <td>:</td>
        <td style="text-align: left">{self.glasses_power:.2f} D</td>
   </tr>
   <tr>
        <td style="text-align: left">Eye model</td>
        <td>:</td>
        <td style="text-align: left">{self.eye_model}</td>
   </tr>
</table>
"""
        )

In [5]:
# define functions to calculate magnification based on user input of eye geometry for different types of eyes


# phakic
def phakic_magnification(
    R_corF=widgets.BoundedFloatText(
        value=7.72,
        min=0,
        description=("Anterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_corB=widgets.BoundedFloatText(
        value=6.50,
        min=0,
        description=("Posterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensF=widgets.BoundedFloatText(
        value=10.2,
        min=0,
        description=("Anterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensB=widgets.BoundedFloatText(
        value=6.0,
        min=0,
        description=("Posterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_cor=widgets.BoundedFloatText(
        value=0.55,
        min=0,
        description=("Corneal thickness (mm)"),
        style={"description_width": "initial"},
        step=0.05,
    ),
    D_ACD=widgets.BoundedFloatText(
        value=3.05,
        min=0,
        description=("Anterior chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.1,
    ),
    D_lens=widgets.BoundedFloatText(
        value=4.00,
        min=0,
        description=("Crystalline lens thickness (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_vitr=widgets.BoundedFloatText(
        value=16.3203,
        min=0,
        description=("Vitreous chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
    SE=widgets.FloatText(
        description=("Spherical eq. of refraction (D)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
):
    phakic_geometry = {
        "R_corF": R_corF * -(10**-3),
        "R_corB": R_corB * -(10**-3),
        "R_lensF": R_lensF * -(10**-3),
        "R_lensB": R_lensB * 10**-3,
        "D_cor": D_cor * 10**-3,
        "D_ACD": D_ACD * 10**-3,
        "D_lens": D_lens * 10**-3,
        "D_vitr": D_vitr * 10**-3,
        "SE": SE,
    }

    # Define a phakic eye model
    phakic_eye = Eye(
        name="phakic",
        geometry=phakic_geometry,
        NType="Navarro",
        refraction=phakic_geometry["SE"],
    )

    phakic_magnification_data = MagnificationData.calculate_from_eye_model(
        phakic_eye, phakic_geometry["SE"]
    )

    display(phakic_magnification_data.to_html())


# pseudophakic
def iol_magnification(
    R_corF=widgets.BoundedFloatText(
        value=7.72,
        min=0,
        description=("Anterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_corB=widgets.BoundedFloatText(
        value=6.50,
        min=0,
        description=("Posterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensF=widgets.BoundedFloatText(
        value=8.16,
        min=0,
        description=("Anterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensB=widgets.BoundedFloatText(
        value=11.18,
        min=0,
        description=("Posterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_cor=widgets.BoundedFloatText(
        value=0.55,
        min=0,
        description=("Corneal thickness (mm)"),
        style={"description_width": "initial"},
        step=0.05,
    ),
    D_ACD=widgets.BoundedFloatText(
        value=3.05,
        min=0,
        description=("Anterior chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.1,
    ),
    D_lens=widgets.BoundedFloatText(
        value=1.00,
        min=0,
        description=("Lens thickness (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_vitr=widgets.BoundedFloatText(
        value=19.3203,
        min=0,
        description=("Vitreous chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
    SE=widgets.FloatText(
        value=0,
        description=("Spherical eq. of refraction (D)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
):
    iol_geometry = {
        "R_corF": R_corF * -(10**-3),
        "R_corB": R_corB * -(10**-3),
        "R_lensF": R_lensF * -(10**-3),
        "R_lensB": R_lensB * 10**-3,
        "D_cor": D_cor * 10**-3,
        "D_ACD": D_ACD * 10**-3,
        "D_lens": D_lens * 10**-3,
        "D_vitr": D_vitr * 10**-3,
        "SE": SE,
    }

    # Define an eye model with an IOL
    iol_eye = Eye(
        name="IOL",
        geometry=iol_geometry,
        model_type="VughtIOL",
        NType="VughtIOL",
        refraction=iol_geometry["SE"],
    )

    iol_magnification_data = MagnificationData.calculate_from_eye_model(
        iol_eye, iol_geometry["SE"]
    )

    display(iol_magnification_data.to_html())


# pIOL
def piol_magnification(
    R_corF=widgets.BoundedFloatText(
        value=7.72,
        min=0,
        description=("Anterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_corB=widgets.BoundedFloatText(
        value=6.50,
        min=0,
        description=("Posterior corneal curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensF=widgets.BoundedFloatText(
        value=10.2,
        min=0,
        description=("Anterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    R_lensB=widgets.BoundedFloatText(
        value=6.0,
        min=0,
        description=("Posterior lens curvature (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_cor=widgets.BoundedFloatText(
        value=0.55,
        min=0,
        description=("Corneal thickness (mm)"),
        style={"description_width": "initial"},
        step=0.05,
    ),
    D_ACD=widgets.BoundedFloatText(
        value=2.35,
        min=0,
        description=("Anterior chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.1,
    ),
    D_lens=widgets.BoundedFloatText(
        value=4.00,
        min=0,
        description=("Crystalline lens thickness (mm)"),
        style={"description_width": "initial"},
        step=0.25,
    ),
    D_vitr=widgets.BoundedFloatText(
        value=13.5303,
        min=0,
        description=("Vitreous chamber depth (mm)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
    SE=widgets.BoundedFloatText(
        value=0,
        description=("Spherical eq. of refraction (D)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
    pIOL_power=widgets.FloatText(
        value=9,
        description=("pIOL power (D)"),
        style={"description_width": "initial"},
        step=0.5,
    ),
):
    piol_geometry = {
        "R_corF": R_corF * -(10**-3),
        "R_corB": R_corB * -(10**-3),
        "R_lensF": R_lensF * -(10**-3),
        "R_lensB": R_lensB * 10**-3,
        "D_cor": D_cor * 10**-3,
        "D_ACD": D_ACD * 10**-3,
        "D_lens": D_lens * 10**-3,
        "D_vitr": D_vitr * 10**-3,
        "SE": SE,
    }

    piol_data = PhakicIOL(
        power=pIOL_power,
        thickness=0.2e-3,
        refractive_index=1.47,
        lens_distance=0.5e-3,
    )

    # Define an eye model with a phakic IOL
    piol_eye = Eye(
        name="pIOL",
        geometry=piol_geometry,
        NType="Navarro",
        refraction=piol_geometry["SE"],
        pIOL=piol_data,
    )

    piol_magnification_data = MagnificationData.calculate_from_eye_model(
        piol_eye, piol_geometry["SE"]
    )

    display(piol_magnification_data.to_html())

In [6]:
# calculate magnification based on user input of eye type
def magnification_calculation(
    type_eye=widgets.Dropdown(
        options=["phakic", "pseudophakic", "pIOL"], description="Eye model"
    )
):
    display(
        HTML(
            f"""
<table>
    <tr>
        <td style="text-align: left">Camera condenser lens power</td>
        <td>:</td>
        <td style="text-align: left">38 D</td>
   </tr>
   <tr>
        <td style="text-align: left">First order correction term</td>
        <td>:</td>
        <td style="text-align: left">-{round(0.0105,2)}</td>
   </tr>
</table>
"""
        )
    )

    if type_eye == "phakic":
        magnification_calculation = widgets.interact_manual(phakic_magnification)
        magnification_calculation.widget.children[9].description = "Calculate"

    if type_eye == "pseudophakic":
        magnification_calculation = widgets.interact_manual(iol_magnification)
        magnification_calculation.widget.children[9].description = "Calculate"

    if type_eye == "pIOL":
        magnification_calculation = widgets.interact_manual(piol_magnification)
        magnification_calculation.widget.children[10].description = "Calculate"


eye_model_choice = widgets.interact_manual(
    magnification_calculation, layout=widgets.Layout(width="500px")
)
eye_model_choice.widget.children[1].description = "Choose eye model"

interactive(children=(Dropdown(description='Eye model', options=('phakic', 'pseudophakic', 'pIOL'), value='pha…